# Use Python tools to process chemical structures of neutral organic molecules.
- Select the molecule to study from [Pubchem](https://pubchem.ncbi.nlm.nih.gov/) database, which contains 111.5 million compounds.

Execute the cell and follow the steps, it is necessary to connect with the user account in which the files are contained.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- The selected structure is the LEVOTHYROXINE [Pubchem](https://pubchem.ncbi.nlm.nih.gov/compound/5819) molecule. The 3D structure of the molecule was extracted and saved in sdf format.


In [ ]:
# Path of the .sdf file with the structure of the selected molecule.
str_archivo = '/content/drive/MyDrive/python para química/MÓDULO 3/Tarea 3/Conformer3D_CID_5819.sdf'

In [ ]:
# Path of the file in which the atomic charges will be stored.
path_to_cargas_file = 'Change for your path'
ruta_archivo_cargas = f"{path_to_cargas_file}cargas.csv"

In [ ]:
# Path of the file in which the coordinates will be stored.
path_to_coordenadas_file = 'Change for your path'
ruta_archivo_coordenadas = f"{path_to_coordenadas_file}coordenadas.xyz"

# Import Statements

In [ ]:
#To obtain the atomic masses, we will use the qcelemental modulus
!pip install qcelemental 
import qcelemental as qcel
import numpy as np
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Process the sdf file with python functions saving the coordinates in a numpy array and the element symbols in a list.
- Get the partial charges of the atoms in the file and save these to a list.
Through the genfromtxt module, we access the file where they are stored, we will obtain the coordinates, along with the atomic symbol of each element involved.
- The first thing will be to obtain the total number of atoms that make up the molecule, which will allow us to understand the total number of lines that we must extract, from where we will obtain the coordinates.

In [ ]:
num_atomos = int(np.genfromtxt(str_archivo, skip_header=3,invalid_raise=False, usecols=0, max_rows= 1))
coordenadas_atomicas = np.genfromtxt(str_archivo, skip_header=4, invalid_raise=False, usecols=(0,1,2), max_rows= num_atomos)

In [ ]:
#coordenadas_atomicas

Using the same method used above, we will also extract the atomic symbols, which will be stored in a list.

In [ ]:
atoms_array = (np.genfromtxt(str_archivo, skip_header=4, invalid_raise=False, dtype=str, usecols=(3), max_rows= num_atomos))
simbolo_atomico = atoms_array.tolist()

In [ ]:
#simbolo_atomico

# Obtaining the partial charges of atoms. 
To extract these loads, it is necessary to access the section within the file named "PUBCHEM_MMFF94_PARTIAL_CHARGES",
To accomplish this, we use the readline() method which accesses the file content line by line.
When the desired section is found, the partial loads are obtained.

In [ ]:
# Function to sort a list of tuples using the sorted() method.
def Ordenar_Tupla(tup):
    return(sorted(tup, key = lambda x: x[0])) 
 

In [ ]:
myfile = open(str_archivo, "r")
myline = myfile.readline()
NUM_PARTIAL_CHARGES = 0
while myline:
  if myline.strip() == '> <PUBCHEM_MMFF94_PARTIAL_CHARGES>':
    NUM_PARTIAL_CHARGES = int(myfile.readline())
    break
  myline = myfile.readline()  

cargas_array =np.genfromtxt(myfile, invalid_raise=False, dtype=[('atomoint','i8'),('cargafloat','f8')], usecols=(0,1), max_rows= NUM_PARTIAL_CHARGES)
cargas_parciales = cargas_array.tolist()

In [ ]:
#cargas_parciales

In [ ]:
# printing the sorted list of tuples
sorted_cargas_parciales = Ordenar_Tupla(cargas_parciales)

In [ ]:
sorted_cargas_parciales

[(1, -0.08),
 (2, -0.08),
 (3, -0.08),
 (4, -0.08),
 (5, -0.17),
 (6, -0.65),
 (7, -0.57),
 (8, -0.53),
 (9, -0.99),
 (10, 0.14),
 (11, 0.33),
 (12, -0.14),
 (13, -0.15),
 (14, -0.15),
 (15, 0.08),
 (16, 0.08),
 (17, 0.08),
 (18, 0.66),
 (19, 0.08),
 (20, -0.15),
 (21, -0.15),
 (22, 0.08),
 (23, 0.08),
 (24, 0.08),
 (28, 0.15),
 (29, 0.15),
 (30, 0.36),
 (31, 0.36),
 (32, 0.15),
 (33, 0.15),
 (34, 0.5),
 (35, 0.45)]

# Molecule transformation
- Write for each atom in the molecule a line with the element symbol of each atom separated by a comma from its charge to a file named cargas.csv.
- Read the cargas.csv file with the pandas module and determine the maximum and minimum atomic charge.

In [ ]:
len_cargas = len(sorted_cargas_parciales)
indices = [sorted_cargas_parciales[i][0] for i in range(0, len_cargas)]
cargas_atomos = {
    'Elementos':  [simbolo_atomico[sorted_cargas_parciales[i][0]-1] for i in range(0, len_cargas)],
    'Cargas parciales':  [sorted_cargas_parciales[i][1] for i in range(0, len_cargas)],
}
cargas = pd.DataFrame(data=cargas_atomos, index=indices)

In [ ]:
cargas.to_csv(ruta_archivo_cargas)

In [ ]:
data_cargas = pd.read_csv(ruta_archivo_cargas)

In [ ]:
print(f"Carga atómica Máxima: {data_cargas['Cargas parciales'].max()}")
print(f"Carga atómica Mínima: {data_cargas['Cargas parciales'].min()}")

Carga atómica Máxima: 0.66
Carga atómica Mínima: -0.99


In [ ]:
#np.around: Round evenly to the given number of decimal places.
coordenadas_atomicas_redondeadas = np.around(coordenadas_atomicas, 3)

In [ ]:
#In a file named coordinates.xyz write in the first line the total number of atoms and then a line with the name of the molecule followed by lines,
#of which each contain the element symbol for each atom and its x,y, and z coordinates to only 3 decimal places.
archivo_escritura = open(ruta_archivo_coordenadas,'w')
archivo_escritura.write(f"Número total de átomos: {num_atomos}\n")
archivo_escritura.write(f"Nombre de la molécula: Levotiroxina\n")
for i in range(num_atomos):
    archivo_escritura.write(f"{simbolo_atomico[i]}  {coordenadas_atomicas_redondeadas[i][0]}  {coordenadas_atomicas_redondeadas[i][1]}  {coordenadas_atomicas_redondeadas[i][2]}\n")
archivo_escritura.close()

In [ ]:
# Get the mass of each atom.
qcel.periodictable.to_mass('I', return_decimal=True)
masas_atomicas = []
for elemento in simbolo_atomico:
  masas_atomicas.append(float(qcel.periodictable.to_mass(elemento, return_decimal=True)))
masas_atomicas

[126.9044719,
 126.9044719,
 126.9044719,
 126.9044719,
 15.99491461957,
 15.99491461957,
 15.99491461957,
 15.99491461957,
 14.00307400443,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 12.0,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223,
 1.00782503223]

In [ ]:
archivo_corrdenadas = open(ruta_archivo_coordenadas,'r')
archivo_corrdenadas.readlines()

['Número total de átomos: 35\n',
 'Nombre de la molécula: Levotiroxina\n',
 'I  0.146  -0.702  -3.237\n',
 'I  0.634  -2.54  2.562\n',
 'I  6.594  -0.232  -0.419\n',
 'I  1.879  3.328  1.109\n',
 'O  1.439  -1.802  -0.484\n',
 'O  -3.201  2.17  -0.586\n',
 'O  -5.395  1.84  -1.056\n',
 'O  4.913  2.359  0.544\n',
 'N  -5.694  0.624  1.386\n',
 'C  -4.075  -0.851  0.282\n',
 'C  -4.374  0.578  0.76\n',
 'C  -2.599  -1.105  0.078\n',
 'C  -1.841  -1.583  1.134\n',
 'C  -2.034  -0.855  -1.162\n',
 'C  0.105  -1.571  -0.298\n',
 'C  -0.672  -1.09  -1.351\n',
 'C  -0.479  -1.817  0.944\n',
 'C  -4.404  1.571  -0.389\n',
 'C  2.309  -0.759  -0.226\n',
 'C  1.802  0.456  0.203\n',
 'C  3.668  -0.954  -0.402\n',
 'C  4.547  0.098  -0.142\n',
 'C  2.68  1.507  0.462\n',
 'C  4.053  1.329  0.29\n',
 'H  -4.616  -1.072  -0.648\n',
 'H  -4.443  -1.584  1.014\n',
 'H  -3.645  0.921  1.503\n',
 'H  -2.309  -1.768  2.097\n',
 'H  -2.652  -0.483  -1.975\n',
 'H  -5.709  0.005  2.196\n',
 'H  -6.395  0

In [ ]:
coordenadas_atomicas = np.genfromtxt(ruta_archivo_coordenadas, skip_header=2, invalid_raise=False, usecols=(1,2,3), max_rows= num_atomos)

In [ ]:
#Function that returns the center of mass. To calculate it, the numpy.dot() function is used, which calculates the dot product of two input arrays.
def computarCOM(mass, coordinates):
      '''
      Retorna el centro de masa.
      '''
      com = np.dot(mass, coordinates) / np.sum(mass)
      return com

In [ ]:
#We get the center of mass using the function created above.
print(f"Center of mass: {computarCOM(masas_atomicas,coordenadas_atomicas)}")

Center of mass: [1.31362192 0.00331617 0.00234582]
